In [1]:
import json
sensordata = []
for line in open('sensor.txt', 'r'):
    sensordata.append(json.loads(line))
GyroScope = []
Accelerometer = []
sa1 = []
sa2 = []
sa3 = []
sg1 = []
sg2 = []
sg3 = []
Gyros = []
asscelo = []
for x in sensordata:
    if x['Sensor_Type'] == 'GyroScope':
        sg1.append(x['val1'])
        sg2.append(x['val2'])
        sg3.append(x['val3'])
        Gyros.append(x['val1'])
        Gyros.append(x['val2'])
        Gyros.append(x['val3'])
    else:
        sa1.append(x['val1'])
        sa2.append(x['val2'])
        sa3.append(x['val3'])
        asscelo.append(x['val1'])
        asscelo.append(x['val2'])
        asscelo.append(x['val3'])

print(sa1)
print(sa2)
print(sa3)
from sklearn.preprocessing import StandardScaler


    

[0.31800001859664917, 0.4129500091075897, 0.44895002245903015, 0.43695002794265747, 0.44505003094673157, 0.5020500421524048, 0.5380499958992004, 0.6150000095367432, 0.7180500626564026, 0.6700500249862671, 0.5910000205039978, 0.5430000424385071, 0.44895002245903015, 0.35100001096725464, 0.27195000648498535, -0.05700000375509262, -0.17505000531673431, -0.1099500060081482, 0.032999999821186066, -0.15105000138282776, -0.2970000207424164, -0.46905001997947693, -0.8520000576972961, -1.111050009727478, -1.3690500259399414, -1.612950086593628, -2.133000135421753, -2.1880500316619873, -2.11395001411438, -1.9800001382827759, -1.815000057220459, -1.0459500551223755, -0.6270000338554382, -0.7560000419616699, -1.2850500345230103, -1.5679501295089722, -1.8600001335144043, -2.302950143814087, -2.509050130844116, -2.3749501705169678, -2.110949993133545, -1.590000033378601, -1.5799500942230225, -2.4589500427246094, -1.9200000762939453, -0.9120000600814819, -0.17505000531673431, -0.5670000314712524, -0.

In [2]:
from statistics import mean
x1  = mean(sa1)
x2  = mean(sa2)
x3  = mean(sa3)
y1  = mean(sg1)
y2  = mean(sg2)
y3  = mean(sg3)
print(x1,x2,x3,y1,y2,y3)

g0 = 9.81
g1 = 9.81
g2 = 9.81
alpha = 0.8
g0 = alpha * g0 + (1 - alpha) * x1;
g1 = alpha * g1 + (1 - alpha) * x2;
g2 = alpha * g2 + (1 - alpha) * x3;


lx1 = x1 - g0;
lx2 = x2 - g1;
lx3 = x3 - g2;


-0.44171332903484867 7.375534019568154 6.24329887987465 -0.031035979664758653 0.014371351179087886 -0.047644492884393076


In [3]:
print(lx1,lx2,lx3)

-8.20137066322788 -1.9475727843454775 -2.8533608961002814


In [4]:
import pandas as pd
import numpy as np
#import standardscaler and label encoder for normalization of data and encoding of class
from sklearn.preprocessing import standardscaler
from sklearn.preprocessing import LabelEncoder

from sklearn.tree import DecisionTreeClassifier as dt
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

ImportError: cannot import name 'standardscaler' from 'sklearn.preprocessing' (/home/da-emon/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/__init__.py)

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

training_activity = train['Activity']
testing_activity = test['Activity']
train = train.iloc[:,:260]
test = test.iloc[:,:260]
#filter data
train = train.drop(train.filter(regex='meanFreq|Jerk|Mag'),axis=1)
test = test.drop(test.filter(regex='meanFreq|Jerk|Mag'),axis=1)

train = train.filter(regex='mean')
test = test.filter(regex='mean')
##print(train)
##print(test)

In [29]:
#now scale the data between 1 to -1
scaler = MinMaxScaler().fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

#now encode the activity
le = LabelEncoder().fit(['STANDING','SITTING','LAYING','WALKING','WALKING_UPSTAIRS','WALKING_DOWNSTAIRS'])
training_act_enc = le.transform(training_activity)
testing_act_enc = le.transform(testing_activity)

print(training_act_enc)
print(testing_act_enc)

[2 2 2 ... 5 5 5]
[2 2 2 ... 5 5 5]


In [30]:
#train the model
svm = SVC(kernel='linear')
dt = dt()
xt = ExtraTreesClassifier()
rf = rf()

kf = KFold(n_splits=10)

def evaluate_model(model):
    score = cross_val_score(model,train,training_act_enc,cv=kf)
    average = score.mean()
    return "Performance : {} \n Accuracy:{}".format(score, average)

print("ExtraTree Classifier : {}".format(evaluate_model(xt)))
print("Random Forest Classifier : {}".format(evaluate_model(rf)))

ExtraTree Classifier : Performance : [0.80298913 0.79619565 0.84081633 0.73605442 0.67210884 0.63537415
 0.77959184 0.75102041 0.78367347 0.76598639] 
 Accuracy:0.7563810632948831
Random Forest Classifier : Performance : [0.80434783 0.80298913 0.85714286 0.7537415  0.67210884 0.72517007
 0.78367347 0.74829932 0.80544218 0.73197279] 
 Accuracy:0.7684887976929902


In [31]:
#Extra Tree Classifier has greater accuracy than all of other. so we will further improve its performance by
#tuning the algorithm
from sklearn.model_selection import RandomizedSearchCV as randomcv

parameters = {'n_estimators':np.arange(20,200,20), 'min_samples_split':np.arange(2,10,2)}
model = ExtraTreesClassifier()
search = randomcv(model,param_distributions=parameters,n_iter=30,n_jobs=4)

search.fit(train,training_act_enc)
print(search.best_score_)
print(search.best_estimator_)

0.7625086363573301
ExtraTreesClassifier(min_samples_split=4, n_estimators=160)


In [32]:
model = search.best_estimator_
model.fit(train,training_act_enc)
prediction = model.predict(test)

In [36]:

import numpy as np
data = np.array([[lx1,lx2,lx3,x1,x2,x3,y1,y2,y3]])
print(data)
data = scaler.transform(data)
findalpredict = model.predict(data)


[[-7.79622865e+00 -7.87221767e+00  6.56898556e-02  6.47141907e-02
  -3.02720814e-02  9.89211232e+00  1.63241923e-06  1.22682644e-04
  -4.65829906e-03]]


In [37]:
print(findalpredict)

[0]


In [38]:
print(data)

[[-1.14874931e+02 -1.92476244e+02  3.08717553e+00 -1.16000625e+00
  -1.10899525e-01  2.81744403e+01  1.49252452e-01  5.76320231e-01
  -6.55053963e-01]]
